# The Devil and the Coin Flip Game

>You're playing a game with the devil, with your soul at stake. You're sitting at a circular table which has 4 coins, arranged in a diamond, at the 12, 3, 6, and 9 o'clock positions. You are blindfolded, and can never see the coins or the table.

>Your goal is to get all 4 coins showing heads, by telling the devil the position(s) of some coins to flip. We call this a "move" on your part. The devil must faithfully perform the requested flips, but may first sneakily rotate the table any number of quarter-turns, so that the coins are in different positions. You keep making moves, and the devil keeps rotating and flipping, until all 4 coins show heads.

> Example: You tell the devil tthe 12 o'clock and 6 o'clock positions. The devil could leave the table unrotated (or could rotate it a half-turn), and then flip the two coins that you specified. Or the devil could rotate the table a quarter turn in either direction, and then flip the coins that are now in the 12 o'clock and 6 o'clock positions (which were formerly at 3 o'clock and 9 o'clock).  You won't know which of these actions the devil took.

> What is a shortest sequence of moves that is *guaranteed* to win, no matter what the initial state of the coins, and no matter what rotations the devil applies?

# Analysis

The player, being blindfolded, does not know the true state of the coins. So the player should represent what is known: the *set of possible states* of the coins. We call this a *belief state*. At the start of the game, each of the four coins could be either heads or tails, so that's 2<sup>4</sup> = 16 possibilities in the belief state. 

A move updates the belief state as follows: for every coin sequence in the current belief state, rotate it in every possible way, and then flip the coins specified by the position(s) in the move. Collect all these results together to form the new belief state. Solving the game means coming up with a sequence of moves that ends in a belief state consisting of just `{'HHHH'}`. It must be a shortest possible sequence, so a breadth-first search seems right. The search space will be tiny, so compute time will be trivial; the only issue is specifying the domain correctly. To increase the chance of getting it correct, I won't try to do anything fancy, such as noticing that some coin sequences are rotational variants of other sequences.


# Implementation Choices

Here are the main concepts, and my implementation choices:

- `Coins`: A *coin sequence* is represented as a `str` of four characters, such as `'HTTT'`. 
- `Belief`: A *belief state* is represented as a `frozenset` of `Coins` (frozen so that it can be hashed in a `set`).
- `Move`: A *move* is a set of positions to flip. A position will be an integer index into the coin sequence, so a move is a set of these such as `{0, 2}`, which we can interpret as "flip the 12 o'clock and 6 o'clock positions." 
- `all_coins`: Set of all possible coin sequences: `{'HHHH', 'HHHT', ...}`.
- `rotations`: The function `rotations(coins)` returns the set of all 4 rotations of the coin sequence.
- `update`: The function `update(belief, move)` retuns an updated belief state, representing all the possible coin sequences that could result from any devil rotation followed by the specified flip(s). (But don't flip `'HHHH'`, because the game would have already ended.)
- `flip`: The function `flip(coins, move)` flips the specified positions within the coin sequence.

In [1]:
from collections import deque, Counter
from itertools   import product, combinations
import random

Coins     = ''.join   # A coin sequence, such as 'HHHT'.
Belief    = frozenset # A belief state is a set of possible coin sequences.
all_coins = Belief(map(Coins, product('HT', repeat=4)))

def rotations(coins) -> [Coins]: 
    "A list of all possible rotations of a coin sequence."
    return [coins[r:] + coins[:r] for r in range(4)]

def update(belief, move) -> Belief:
    "Update belief: consider all possible rotations, then flip."
    return Belief((c if c == 'HHHH' else flip(c, move))
                  for coins in belief
                  for c in rotations(coins))

def flip(coins, move) -> Coins:
    "Flip the coins in the positions specified by the move."
    coins = list(coins) # Need a mutable sequence
    for i in move:
        coins[i] = ('H' if coins[i] == 'T' else 'T')
    return Coins(coins)

Let's try out these functions:

In [2]:
flip('HHHT', {0, 2})

'THTT'

In [3]:
rotations('HHHT')

['HHHT', 'HHTH', 'HTHH', 'THHH']

In [4]:
all_coins

frozenset({'HHHH',
           'HHHT',
           'HHTH',
           'HHTT',
           'HTHH',
           'HTHT',
           'HTTH',
           'HTTT',
           'THHH',
           'THHT',
           'THTH',
           'THTT',
           'TTHH',
           'TTHT',
           'TTTH',
           'TTTT'})

We see there are 16 coin sequences in the `all_coins` belief state. Now if we update this belief state by flipping all 4 positions, we should get a new belief state where we have eliminated the possibility of 4 tails, leaving 15 possible coin sequences:

In [5]:
update(all_coins, {0, 1, 2, 3})

frozenset({'HHHH',
           'HHHT',
           'HHTH',
           'HHTT',
           'HTHH',
           'HTHT',
           'HTTH',
           'HTTT',
           'THHH',
           'THHT',
           'THTH',
           'THTT',
           'TTHH',
           'TTHT',
           'TTTH'})

Everything looks good so far. One more thing: we need to find all subsets of the 4 positions; these are the possible moves:

In [6]:
def powerset(sequence): 
    "All subsets of a sequence."
    return [set(c) 
            for r in range(len(sequence) + 1)
            for c in combinations(sequence, r)]

In [7]:
powerset(range(4))

[set(),
 {0},
 {1},
 {2},
 {3},
 {0, 1},
 {0, 2},
 {0, 3},
 {1, 2},
 {1, 3},
 {2, 3},
 {0, 1, 2},
 {0, 1, 3},
 {0, 2, 3},
 {1, 2, 3},
 {0, 1, 2, 3}]

# Search for a Solution

The function `search` does a breadth-first search starting
from a `belief` state consisting of `all_coins`, and applying a sequences of moves, trying to
find a sequence that leads to the goal belief state in which the only possibility is 4 heads: `{'HHHH'}`.
As is typical for search algorithms, we build a search tree, keeping a queue of tree `nodes` to consider, where each 
node consists of a path (a sequence of moves) and a resulting belief state. We also keep track, in `explored`, of
the states we have already explored, so that we don't have to revisit them.

In [8]:
def search():
    "Breadth-first search from all_coins to all heads."
    explored = set()
    queue = deque([Node([], all_coins)])
    while queue:
        (path, belief) = queue.popleft()
        if belief == {'HHHH'}:
            return path
        for move in powerset(range(4)):
            belief2 = update(belief, move)
            if belief2 not in explored:
                explored.add(belief2)
                queue.append(Node(path + [move], belief2))
                
def Node(path, belief) -> tuple: return (path, belief)

# A Solution

In [9]:
search()

[{0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0, 1},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0, 1},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3}]

That's a 15-move sequence that is guaranteed to lead to a win. You can stop reading here if all you want is the answer to the puzzle. Or you can continue on ...

# Verifying the Solution

Can I verify that the solution is correct?  Exploring with paper and pencil, it does appear to work. A colleague did the puzzle and got the same answer. And here's further validation: The function `random_devil` takes an initial coin sequence and a sequence of moves, and plays those moves with a devil that chooses rotations randomly, returning the number of moves it takes until the player wins:

In [10]:
def random_devil(coins, moves) -> int or None:
    """A random devil responds to moves starting from coins; 
    return the number of moves until win, or None."""
    if coins == 'HHHH': return 0
    for (i, move) in enumerate(moves, 1):
        coins = flip(random.choice(list(rotations(coins))), move)
        if coins == 'HHHH': 
            return i

I will let the `random_devil` play 1000 times from each possible starting coin sequence:

In [11]:
moves = search()

Counter(random_devil(coins, moves) 
        for coins in all_coins
        for _ in range(1000))

Counter({0: 1000,
         1: 1000,
         2: 986,
         3: 1014,
         4: 1052,
         5: 1004,
         6: 976,
         7: 968,
         8: 964,
         9: 1014,
         10: 985,
         11: 1003,
         12: 1056,
         13: 983,
         14: 1021,
         15: 974})

This says that the player won all 16,000 times. (If the player had ever lost, there would have been an entry for `None` in the Counter.)
The remarkable thing, which I can't explain, is that there are very nearly exactly 1,000 results for each of the move counts from 0 to 15. Can you explain that?

# Canonical Coin Sequences and Moves

Consider the four coin sequences `{'HHHT', 'HHTH', 'HTHH', 'THHH'}`. In a sense, these are all the same: they all denote the same sequence of coins with the table rotated to different degrees. Since the devil is free to rotate the table any amount at any time, we could be justified in treating all four of these as equivalent, and collapsing them into one representative member (we could arbitrarily choose the one that comes first in alphabetical order, `'HHHT'`). I will write a definition for `Belief` that returns a `frozenset`, just like before, but makes it a set of canonical coin sequences.

Similarly, the four moves `[{3}, {2}, {1}, {0}]` are equivalent, in that they all say "flip one of the coins, but since you're going to rotate the table first, it doesn't matter which one I specify." I will write a function `canonical_moves` that lists all the canonical rotationally invariant moves corresponding to a belief state.

In [12]:
def Belief(coin_collection): 
    "A set of all the coin sequences in this collection, canonicalized."
    return frozenset(min(rotations(c)) for c in coin_collection)

def canonical_moves(coin_collection):
    "All rotationally invariant moves for a sequence of N coins."
    return [set(i for (i, coin) in enumerate(coins) if coin == 'H' and 'H')
            for coins in sorted(coin_collection) if 'H' in coins]

In [13]:
Belief(all_coins)

frozenset({'HHHH', 'HHHT', 'HHTT', 'HTHT', 'HTTT', 'TTTT'})

In [14]:
canonical_moves(_)

[{0, 1, 2, 3}, {0, 1, 2}, {0, 1}, {0, 2}, {0}]

The starting belief set is down from 16 sequences to 6. Call these  4 heads, 3 heads, 2 adjacent heads, 2 opposite heads, 1 head, and no heads, respectively. Similarly, there are only 5 distinct moves, which we can call flip 4, flip 3, flip 2 adjacent, flip 2 opposite, and flip 1.

I have to confess to a form of cheating here: I threw in the `sorted` in `canonical_moves` even though it was not necessary. By including it, I made sure that `{0, 1, 2, 3}` is the first of the possible moves, which makes `search` faster, because half the time `{0, 1, 2, 3}` is the right move to make.

# Visualizing the Solution

How does the solution work? One answer is "it takes care of all possibilities." But it would be nice to gain more insight. I'll print a table showing the belief state after each move, using the newly-defined canonicalized `Belief` form, and lining them up neatly in columns.

In [15]:
def show(moves, belief=all_coins):
    "For each move, print the move number, move, and belief state."
    show_line(0, 'start', belief)
    for (i, move) in enumerate(moves, 1):
        belief = update(belief, move)
        show_line(i, move, belief)

def show_line(i, move, belief, order=sorted(Belief(all_coins))):
    "Print the move number, move, and belief state."
    ordered_belief = [(coins if coins in belief else '    ')
                      for coins in order]
    print('{:2} | {:5} | {} | {}'
          .format(i, join(move), join(ordered_belief, ' '), i))
    
def join(items, sep='') -> str: return sep.join(map(str, items))

In [16]:
show(search())

 0 | start | HHHH HHHT HHTT HTHT HTTT TTTT | 0
 1 | 0123  | HHHH HHHT HHTT HTHT HTTT      | 1
 2 | 02    | HHHH HHHT HHTT      HTTT TTTT | 2
 3 | 0123  | HHHH HHHT HHTT      HTTT      | 3
 4 | 01    | HHHH HHHT      HTHT HTTT TTTT | 4
 5 | 0123  | HHHH HHHT      HTHT HTTT      | 5
 6 | 02    | HHHH HHHT           HTTT TTTT | 6
 7 | 0123  | HHHH HHHT           HTTT      | 7
 8 | 0     | HHHH      HHTT HTHT      TTTT | 8
 9 | 0123  | HHHH      HHTT HTHT           | 9
10 | 02    | HHHH      HHTT           TTTT | 10
11 | 0123  | HHHH      HHTT                | 11
12 | 01    | HHHH           HTHT      TTTT | 12
13 | 0123  | HHHH           HTHT           | 13
14 | 02    | HHHH                     TTTT | 14
15 | 0123  | HHHH                          | 15


We can see that every odd-numbered move flips all four coins to eliminate the possibility of `TTTT`, flipping it to `HHHH`. We can also see that moves 2, 4, and 6 flip two coins and have the effect of eventually eliminating the two "two heads" sequences from the belief state, and then move 8 flips a single coin, thus eliminating the "three heads" and "one heads" sequences, and bringing back the "two heads" possibilities. Repeating moves 2, 4, and 6 in moves 10, 12, and 14 then re-eliminates the "two heads", and move 15 gets the belief state down to `{'HHHH'}`.



# Different Numbers of Coins

What if there are 3 coins on the table arranged in a triangle? Or 6 coins in a hexagon? To answer that, I'll generalize the two functions that have a "4" in them, `search`, and `rotations`.  I'll also generalize `update`, which had `'HHHH'` in it. And I'll introduce the function `canonical_coins` to return a belief state of all canonical coin sequences of length `N`.

In [17]:
def search(N=4):
    "Breadth-first search from all_coins to all heads."
    start     = canonical_coins(N)
    all_moves = canonical_moves(start)
    goal      = {'H' * N}
    explored  = set()
    queue     = deque([Node([], start)])
    while queue:
        (path, belief) = queue.popleft()
        if belief == goal:
            return path
        for move in all_moves:
            belief2 = update(belief, move)
            if belief2 not in explored:
                explored.add(belief2)
                queue.append(Node(path + [move], belief2))

def rotations(coins) -> [Coins]: 
    "A list of all possible rotations of a coin sequence."
    return [coins[r:] + coins[:r] for r in range(len(coins))]
                       
def update(belief, move) -> Belief:
    "Update belief: consider all possible rotations, then flip."
    return Belief((flip(c, move) if 'T' in c else c)
                  for coins in belief
                  for c in rotations(coins))

def canonical_coins(N) -> Belief: return Belief(map(Coins, product('HT', repeat=N)))

First verify that we didn't break `search`:

In [18]:
search(4)

[{0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0, 1},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0, 1, 2},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0, 1},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3}]

Now test the new definitions:

In [19]:
rotations('HHHHHT')

['HHHHHT', 'HHHHTH', 'HHHTHH', 'HHTHHH', 'HTHHHH', 'THHHHH']

In [20]:
canonical_coins(4)

frozenset({'HHHH', 'HHHT', 'HHTT', 'HTHT', 'HTTT', 'TTTT'})

In [21]:
canonical_coins(5)

frozenset({'HHHHH',
           'HHHHT',
           'HHHTT',
           'HHTHT',
           'HHTTT',
           'HTHTT',
           'HTTTT',
           'TTTTT'})

In [22]:
canonical_moves(_)

[{0, 1, 2, 3, 4}, {0, 1, 2, 3}, {0, 1, 2}, {0, 1, 3}, {0, 1}, {0, 2}, {0}]

In [23]:
{N: len(canonical_coins(N))
 for N in range(1, 11)}

{1: 2, 2: 3, 3: 4, 4: 6, 5: 8, 6: 14, 7: 20, 8: 36, 9: 60, 10: 108}

On the one hand this is encouraging; there are only 108 canonical coin sequences of length 10, far less than the 1024 non-canonical squences. On the other hand, it is discouraging; since we are searching over the belief states, that would be 2<sup>108</sup> ≌ 10<sup>32</sup> belief states, which is infeasible. However, we should be able to easily handle up to N=7, because 2<sup>20</sup> is only a million.

# Solutions for 1 to 7 Coins

In [24]:
{N: search(N) for N in range(1, 8)}

{1: [{0}],
 2: [{0, 1}, {0}, {0, 1}],
 3: None,
 4: [{0, 1, 2, 3},
  {0, 2},
  {0, 1, 2, 3},
  {0, 1},
  {0, 1, 2, 3},
  {0, 2},
  {0, 1, 2, 3},
  {0, 1, 2},
  {0, 1, 2, 3},
  {0, 2},
  {0, 1, 2, 3},
  {0, 1},
  {0, 1, 2, 3},
  {0, 2},
  {0, 1, 2, 3}],
 5: None,
 6: None,
 7: None}

Too bad; there are no solutions for N = 3, 5, 6, or 7.  

There are solutions for N = 1, 2, 4; of lengths 1, 3, 15. That suggests the conjecture: for every *N* that is a power of 2, there will be a solution of length 2<sup>*N*</sup> - 1. 

# Solution for 8 Coins

Even though there are 2<sup>36</sup> = 68 billion belief states fo N = 8, I hope we can find a solution early in the search process. Let's try:

In [25]:
%time solution8 = search(8)
len(solution8)

CPU times: user 1min 47s, sys: 731 ms, total: 1min 48s
Wall time: 1min 50s


255

Eureka! That's evidence in favor of my conjecture, and it took less time than I thought it would. It is still not proof, and I'll need a more efficient/clever approach to test even for N=16. Can you come up with that approach? Can you prove the conjecture for all powers of 2? Here's the solution for N = 8 to help you discover a pattern:

In [26]:
solution8

[{0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 4, 5},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 2, 4, 5, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 4, 5},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 2, 3},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 4, 5},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 2, 4, 5, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 4, 5},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 2, 3, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 4, 5},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 2, 4, 5, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 4, 5},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2,